## Work
1. 試比較 save_best_only 與否的差異
2. 請僅存入將 save_weights_only 設定為 True, 並嘗試 reset ipynb 並將模型與權重重新建回並預測 x_test

In [1]:
import os
import keras

# Disable GPU
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
from keras.layers import BatchNormalization
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:
## 超參數設定
LEARNING_RATE = 1e-3
EPOCHS = 200
BATCH_SIZE = 1024
MOMENTUM = 0.95

In [7]:
# 載入 Callbacks
from keras.callbacks import ModelCheckpoint
model_ckpt = ModelCheckpoint(filepath="./tmp.h5", 
                             monitor="val_acc", 
                             save_best_only=True)

In [8]:
model = build_mlp(input_shape=x_train.shape[1:])
model.summary()
optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

model.fit(x_train, y_train, 
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          validation_data=(x_test, y_test), 
          shuffle=True,
          callbacks=[model_ckpt]
         )
model.save("tmp_model.h5")
model.save_weights("tmp_weights.h5")
# Collect results
train_loss = model.history.history["loss"]
valid_loss = model.history.history["val_loss"]
train_acc = model.history.history["acc"]
valid_acc = model.history.history["val_acc"]

pred_final = model.predict(x_test)
# Load back
model_load = keras.models.load_model("tmp.h5")
predict_load_back = model_load.predict(x_test)

W0401 15:49:03.717323 140503072831296 deprecation.py:506] From /home/dcube/.local/lib/python3.6/site-packages/tensorflow/python/training/moving_averages.py:210: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
batch_normalization_3 (Batch (None, 128)               512       
__________

50000/50000 [==============================] - 1s 14us/step - loss: 0.5120 - acc: 0.8562 - val_loss: 1.6492 - val_acc: 0.4953
Epoch 50/200
50000/50000 [==============================] - 1s 13us/step - loss: 0.4968 - acc: 0.8616 - val_loss: 1.7004 - val_acc: 0.4910
Epoch 51/200
50000/50000 [==============================] - 1s 14us/step - loss: 0.4800 - acc: 0.8687 - val_loss: 1.6842 - val_acc: 0.4893
Epoch 52/200
50000/50000 [==============================] - 1s 13us/step - loss: 0.4662 - acc: 0.8738 - val_loss: 1.6987 - val_acc: 0.4937
Epoch 53/200
50000/50000 [==============================] - 1s 14us/step - loss: 0.4503 - acc: 0.8812 - val_loss: 1.7093 - val_acc: 0.4928
Epoch 54/200
50000/50000 [==============================] - 1s 14us/step - loss: 0.4338 - acc: 0.8866 - val_loss: 1.7409 - val_acc: 0.4852
Epoch 55/200
50000/50000 [==============================] - 1s 13us/step - loss: 0.4213 - acc: 0.8905 - val_loss: 1.7443 - val_acc: 0.4900
Epoch 56/200
50000/50000 [==============

Epoch 108/200
50000/50000 [==============================] - 1s 14us/step - loss: 0.0506 - acc: 0.9986 - val_loss: 2.5507 - val_acc: 0.4695
Epoch 109/200
50000/50000 [==============================] - 1s 13us/step - loss: 0.0483 - acc: 0.9989 - val_loss: 2.5517 - val_acc: 0.4755
Epoch 110/200
50000/50000 [==============================] - 1s 14us/step - loss: 0.0471 - acc: 0.9989 - val_loss: 2.5789 - val_acc: 0.4761
Epoch 111/200
50000/50000 [==============================] - 1s 14us/step - loss: 0.0454 - acc: 0.9989 - val_loss: 2.5688 - val_acc: 0.4744
Epoch 112/200
50000/50000 [==============================] - 1s 13us/step - loss: 0.0446 - acc: 0.9990 - val_loss: 2.5879 - val_acc: 0.4764
Epoch 113/200
50000/50000 [==============================] - 1s 14us/step - loss: 0.0431 - acc: 0.9991 - val_loss: 2.6070 - val_acc: 0.4723
Epoch 114/200
50000/50000 [==============================] - 1s 14us/step - loss: 0.0415 - acc: 0.9991 - val_loss: 2.6036 - val_acc: 0.4785
Epoch 115/200
50000/

50000/50000 [==============================] - 1s 15us/step - loss: 0.0137 - acc: 1.0000 - val_loss: 2.9591 - val_acc: 0.4698
Epoch 167/200
50000/50000 [==============================] - 1s 15us/step - loss: 0.0134 - acc: 1.0000 - val_loss: 2.9628 - val_acc: 0.4692
Epoch 168/200
50000/50000 [==============================] - 1s 14us/step - loss: 0.0129 - acc: 1.0000 - val_loss: 2.9718 - val_acc: 0.4731
Epoch 169/200
50000/50000 [==============================] - 1s 14us/step - loss: 0.0129 - acc: 1.0000 - val_loss: 2.9799 - val_acc: 0.4702
Epoch 170/200
50000/50000 [==============================] - 1s 13us/step - loss: 0.0128 - acc: 1.0000 - val_loss: 2.9760 - val_acc: 0.4710
Epoch 171/200
50000/50000 [==============================] - 1s 14us/step - loss: 0.0127 - acc: 1.0000 - val_loss: 2.9951 - val_acc: 0.4734
Epoch 172/200
50000/50000 [==============================] - 1s 14us/step - loss: 0.0126 - acc: 1.0000 - val_loss: 2.9929 - val_acc: 0.4701
Epoch 173/200
50000/50000 [=======

In [10]:
from sklearn.metrics import accuracy_score
print("load_model:%s"%(accuracy_score(y_test.argmax(axis=-1),predict_load_back.argmax(axis=-1))))
_, acc = model.evaluate(x_test, y_test)
print("EPOCHS:%s, acc:%s"%(EPOCHS,acc))

load_model:0.5032
10000/10000 [==============================] - 0s 33us/step
EPOCHS:200, acc:0.4727
